In [ ]:
import pathlib
import random
import shutil
import sys
import xml.etree.ElementTree as ET


import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import torch
import ultralytics
import yaml
from IPython import display
from PIL import Image
from tqdm.notebook import tqdm
from ultralytics import YOLO

**print nesasary data **

In [2]:
print("Platform:", sys.platform)
print("Python version:", sys.version)
print("---")
print("matplotlib version:", plt.matplotlib.__version__)
print("pandas version:", pd.__version__)
print("PIL version:", Image.__version__)
print("PyYAML version:", yaml.__version__)
print("torch version:", torch.__version__)
print("ultralytics version:", ultralytics.__version__)

Platform: win32
Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
---
matplotlib version: 3.9.2
pandas version: 2.2.2
PIL version: 10.4.0
PyYAML version: 6.0.1
torch version: 2.5.1
ultralytics version: 8.3.78


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")

Using cuda device.


### Data Transformation

In [4]:
!tree id_card_dataset/ --filelimit=3

Too many parameters - --filelimit=3


In [16]:
def parse_annotations(f):
    """Parse all of the objects in a given XML file to YOLO format.

    Input:  f      The path of the file to parse.

    Output: A list of objects in YOLO format.
            Each object is a list [index, x_center, y_center, width, height]."""

    objects = []

    tree = ET.parse(f)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        label = obj.find("name").text
        class_id = class_mapping[label]
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        yolo_bbox = xml_to_yolo_bbox([xmin, ymin, xmax, ymax], width, height)

        objects.append([class_id] + yolo_bbox)

    return objects


objects = parse_annotations(annotations_dir / "01.xml")
print("First object:", objects[0])

FileNotFoundError: [Errno 2] No such file or directory: 'id_card_dataset\\train\\annotations\\01.xml'

In [6]:
training_dir = pathlib.Path("id_card_dataset", "train")
images_dir = training_dir / "images"
annotations_dir = training_dir / "annotations"

print("Images     :", images_dir)
print("Annotations:", annotations_dir)

Images     : id_card_dataset\train\images
Annotations: id_card_dataset\train\annotations


In [24]:
model = YOLO("yolov8n.pt")

model = model.to(device)

In [ ]:
results = model.train(
    data="id_card_dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="id_card_detector_augmented",
    project="runs/train_yolo",
    device=0,
    degrees=10,        # rotate images by ±10°
    scale=0.5,         # zoom in/out
    shear=5.0,         # shearing
    perspective=0.001, # slight perspective distortion
    flipud=0.2,        # vertical flip with 20% chance
    fliplr=0.5,        # horizontal flip with 50% chance
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4  # color jitter
)


In [4]:
"""check trainded yolo model"""

'check trainded yolo model'

In [ ]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt

# Load your trained model
model_2 = YOLO("./models/best.pt")

# Load test image
img_path = "./id_card_dataset/test/images/s63_jpg.rf.385e7719d5b719273015b020eaf97b91.jpg"
results = model_2.(img_path)
# Show results with bounding boxes
results[0].show()

# (Optional) Display with matplotlib in Colab

